In [5]:
!pip3 install pyspark
!pip3 install pandas
!pip3 install pyarrow
!pip3 install install-jdk

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 8.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 7.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 4.8 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [244]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, when, lit, to_date,udf
from pyspark.sql.types import StringType, IntegerType

# NETFLIX DATASET

In [245]:
pd.read_csv('netflix_titles.csv')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [246]:
spark=SparkSession.builder.appName('Practise').getOrCreate()

In [247]:
spark

In [248]:
#Import dataset into pyspark
df_py=spark.read.option('header','true').csv('netflix_titles.csv')

In [249]:
df_py.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [250]:
#CLASS TO CATAGORISE MOVIES INTO SECTIONS

def categorize_duration(duration):
    try:
        duration = int(duration.split(" ")[0])
    except:
        duration = 0
    if duration < 30:
        return "< 30 min"
    elif duration >= 30 and duration < 60:
        return "30-60 min"
    elif duration >= 60 and duration < 120:
        return "1-2 hours"
    else:
        return "> 2 hours"

# Create a UDF to apply the function to the duration column
udf_categorize_duration = udf(categorize_duration, StringType())

In [251]:
#filter out all records which are TV Shows - we’re only interested in films
df_py=df_py.filter("type != 'TV Show'")

#Change the year into the yyyy-mm-dd. date of string format converted to date.
df_py=df_py.withColumn("date_added",to_date(col("date_added"), "MMMM dd, yyyy"))

#rename the date_added column to airing_date
df_py=df_py.withColumnRenamed("date_added", "airing_date")

#new column ‘streaming_service’ to all entries with the value ‘Netflix’
df_py=df_py.withColumn("streaming_service", lit("Netflix"))

#new column ‘adult’ which will be True for all entries with a rating of ‘TV-MA’ or ‘R’ and False otherwise
df_py=df_py.withColumn("adult", when((col("rating") == "TV-MA") | (col("rating") == "R"), True).otherwise(False))

#new column ‘american’ which will be True for entries with a country of ‘United States’ and False for all other values
df_py=df_py.withColumn("american",when(col("country") == "United States", True).otherwise(False))


# Clean the "duration" column
df_py = df_py.withColumn("duration_cleaned", when(col("duration").contains("min"), col("duration")).otherwise("0 min"))

# Apply the UDF to the cleaned "duration" column
df_py = df_py.withColumn("duration_cat", udf_categorize_duration("duration_cleaned"))

#keep only the relevant columns
columns_to_keep = ["title", "director", "airing_date", "country", "adult","american", "streaming_service", "duration_cat"]
df_py = df_py.select(*columns_to_keep)

df_py.show()

#Export to Parquet file
df_py.write.mode("overwrite").parquet("Netflix.parquet")

+--------------------+--------------------+-----------+--------------------+-----+--------+-----------------+------------+
|               title|            director|airing_date|             country|adult|american|streaming_service|duration_cat|
+--------------------+--------------------+-----------+--------------------+-----+--------+-----------------+------------+
|Dick Johnson Is Dead|     Kirsten Johnson| 2021-09-25|       United States|false|    true|          Netflix|   1-2 hours|
|My Little Pony: A...|Robert Cullen, Jo...| 2021-09-24|                null|false|   false|          Netflix|   1-2 hours|
|             Sankofa|        Haile Gerima| 2021-09-24|United States, Gh...| true|   false|          Netflix|   > 2 hours|
|        The Starling|      Theodore Melfi| 2021-09-24|       United States|false|    true|          Netflix|   1-2 hours|
|        Je Suis Karl| Christian Schwochow| 2021-09-23|Germany, Czech Re...| true|   false|          Netflix|   > 2 hours|
|Confessions of 

# AMAZON DATASET

In [252]:
pd.read_csv('amazon_prime_titles.csv')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...
...,...,...,...,...,...,...,...,...,...,...,...,...
9663,s9664,Movie,Pride Of The Bowery,Joseph H. Lewis,"Leo Gorcey, Bobby Jordan",NaN,NaN,1940,7+,60 min,Comedy,New York City street principles get an East Si...
9664,s9665,TV Show,Planet Patrol,NaN,"DICK VOSBURGH, RONNIE STEVENS, LIBBY MORRIS, M...",NaN,NaN,2018,13+,4 Seasons,TV Shows,"This is Earth, 2100AD - and these are the adve..."
9665,s9666,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",NaN,NaN,2008,R,90 min,Action,"In war-torn Eastern Europe, a world-weary grou..."
9666,s9667,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...",NaN,NaN,2021,TV-MA,1 Season,"Drama, Sports","The series tells the story of Diego Maradona, ..."


In [253]:
df=spark.read.option('header','true').csv('amazon_prime_titles.csv')

In [254]:
#filter out all records which are TV Shows - we’re only interested in films
df=df.filter("type != 'TV Show'")

#Used legacy time date settings
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

#Change the year into the yyyy-mm-dd. date of string format converted to date.
df=df.withColumn("date_added",to_date(col("date_added"), "MMMM dd, yyyy"))

#rename the date_added column to airing_date
df=df.withColumnRenamed("date_added", "airing_date")

#new column ‘streaming_service’ to all entries with the value ‘Amazon’
df=df.withColumn("streaming_service", lit("Amazon"))

#new column ‘adult’ which will be True for all entries with a rating of ‘TV-MA’ or ‘R’ and False otherwise
df=df.withColumn("adult", when((col("rating") == "TV-MA") | (col("rating") == "R"), True).otherwise(False))

#new column ‘american’ which will be True for entries with a country of ‘United States’ and False for all other values
df=df.withColumn("american",when(col("country") == "United States", True).otherwise(False))

# Clean the "duration" column
df = df.withColumn("duration_cleaned", when(col("duration").contains("min"), col("duration")).otherwise("0 min"))

# Apply the UDF to the cleaned "duration" column
df = df.withColumn("duration_cat", udf_categorize_duration("duration_cleaned"))

#keep only the relevant columns
columns_to_keep = ["title", "director", "airing_date", "country", "adult","american", "streaming_service", "duration_cat"]
df = df.select(*columns_to_keep)

df.show()

#Export to Parquet file
df.write.mode("overwrite").parquet("Amazon.parquet")

+--------------------+--------------------+-----------+--------------+-----+--------+-----------------+------------+
|               title|            director|airing_date|       country|adult|american|streaming_service|duration_cat|
+--------------------+--------------------+-----------+--------------+-----+--------+-----------------+------------+
| The Grand Seduction|        Don McKellar| 2021-03-30|        Canada|false|   false|           Amazon|   1-2 hours|
|Take Care Good Night|        Girish Joshi| 2021-03-30|         India|false|   false|           Amazon|   1-2 hours|
|Secrets of Deception|         Josh Webber| 2021-03-30| United States|false|    true|           Amazon|   1-2 hours|
|  Pink: Staying True|      Sonia Anderson| 2021-03-30| United States|false|    true|           Amazon|   1-2 hours|
|       Monster Maker|        Giles Foster| 2021-03-30|United Kingdom|false|   false|           Amazon|   30-60 min|
|Living With Dinos...|        Paul Weiland| 2021-03-30|United Ki

# DISNEY DATASET

In [255]:
pd.read_csv('disney_plus_titles.csv')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...
...,...,...,...,...,...,...,...,...,...,...,...,...
1445,s1446,Movie,X-Men Origins: Wolverine,Gavin Hood,"Hugh Jackman, Liev Schreiber, Danny Huston, wi...","United States, United Kingdom","June 4, 2021",2009,PG-13,108 min,"Action-Adventure, Family, Science Fiction",Wolverine unites with legendary X-Men to fight...
1446,s1447,Movie,Night at the Museum: Battle of the Smithsonian,Shawn Levy,"Ben Stiller, Amy Adams, Owen Wilson, Hank Azar...","United States, Canada","April 2, 2021",2009,PG,106 min,"Action-Adventure, Comedy, Family",Larry Daley returns to rescue some old friends...
1447,s1448,Movie,Eddie the Eagle,Dexter Fletcher,"Tom Costello, Jo Hartley, Keith Allen, Dickon ...","United Kingdom, Germany, United States","December 18, 2020",2016,PG-13,107 min,"Biographical, Comedy, Drama","True story of Eddie Edwards, a British ski-jum..."
1448,s1449,Movie,Bend It Like Beckham,Gurinder Chadha,"Parminder Nagra, Keira Knightley, Jonathan Rhy...","United Kingdom, Germany, United States","September 18, 2020",2003,PG-13,112 min,"Buddy, Comedy, Coming of Age",Despite the wishes of their traditional famili...


In [256]:
df_dis=spark.read.option('header','true').csv('disney_plus_titles.csv')

In [257]:

df_dis=df_dis.filter("type != 'TV Show'")

df_dis=df_dis.withColumn("date_added",to_date(col("date_added"), "MMMM dd, yyyy"))

df_dis=df_dis.withColumnRenamed("date_added", "airing_date")

df_dis=df_dis.withColumn("streaming_service", lit("Disney"))

df_dis=df_dis.withColumn("adult", when((col("rating") == "TV-MA") | (col("rating") == "R"), True).otherwise(False))

df_dis=df_dis.withColumn("american",when(col("country") == "United States", True).otherwise(False))

# Clean the "duration" column
df_dis = df_dis.withColumn("duration_cleaned", when(col("duration").contains("min"), col("duration")).otherwise("0 min"))

# Apply the UDF to the cleaned "duration" column
df_dis = df_dis.withColumn("duration_cat", udf_categorize_duration("duration_cleaned"))

columns_to_keep = ["title", "director", "airing_date", "country", "adult","american", "streaming_service", "duration_cat"]
df_dis = df_dis.select(*columns_to_keep)

df_dis.show()

df_dis.write.mode("overwrite").parquet("disney.parquet")

+--------------------+--------------------+-----------+-------------+-----+--------+-----------------+------------+
|               title|            director|airing_date|      country|adult|american|streaming_service|duration_cat|
+--------------------+--------------------+-----------+-------------+-----+--------+-----------------+------------+
|Duck the Halls: A...|Alonso Ramirez Ra...| 2021-11-26|         null|false|   false|           Disney|    < 30 min|
|Ernest Saves Chri...|         John Cherry| 2021-11-26|         null|false|   false|           Disney|   1-2 hours|
|Ice Age: A Mammot...|        Karen Disher| 2021-11-26|United States|false|    true|           Disney|    < 30 min|
|The Queen Family ...|     Hamish Hamilton| 2021-11-26|         null|false|   false|           Disney|   30-60 min|
|   Becoming Cousteau|          Liz Garbus| 2021-11-24|United States|false|    true|           Disney|   1-2 hours|
|A Muppets Christm...|    Kirk R. Thatcher| 2021-11-19|United States|fal

In [258]:
df1 = spark.read.parquet("Netflix.parquet")
df2 = spark.read.parquet("Amazon.parquet")
df3 = spark.read.parquet("disney.parquet")

df1 = df1.select("title", "director", "airing_date", "country", "adult", "american", "streaming_service", "duration_cat")
df2 = df2.select("title", "director", "airing_date", "country", "adult", "american", "streaming_service", "duration_cat")
df3 = df3.select("title", "director", "airing_date", "country", "adult", "american", "streaming_service", "duration_cat")

df_union = df1.union(df2).union(df3)

# Write the merged DataFrame to a parquet file
df_union.write.mode("overwrite").parquet("cleaned_films.parquet")

In [259]:



df4 = spark.read.parquet("cleaned_films.parquet")

df4.filter(df4.streaming_service == "Disney").show()

+--------------------+--------------------+-----------+-------------+-----+--------+-----------------+------------+
|               title|            director|airing_date|      country|adult|american|streaming_service|duration_cat|
+--------------------+--------------------+-----------+-------------+-----+--------+-----------------+------------+
|Duck the Halls: A...|Alonso Ramirez Ra...| 2021-11-26|         null|false|   false|           Disney|    < 30 min|
|Ernest Saves Chri...|         John Cherry| 2021-11-26|         null|false|   false|           Disney|   1-2 hours|
|Ice Age: A Mammot...|        Karen Disher| 2021-11-26|United States|false|    true|           Disney|    < 30 min|
|The Queen Family ...|     Hamish Hamilton| 2021-11-26|         null|false|   false|           Disney|   30-60 min|
|   Becoming Cousteau|          Liz Garbus| 2021-11-24|United States|false|    true|           Disney|   1-2 hours|
|A Muppets Christm...|    Kirk R. Thatcher| 2021-11-19|United States|fal